In [1]:
%load_ext autoreload
%autoreload 1
%aimport asyncio_gpt
import pickle
import json
import pandas as pd
from asyncio_gpt import AsyncioRequestGPT
import asyncio
from tqdm import tqdm
import os

# 读取定义

In [2]:
topic_define_dict = json.load(open("/workspace/spam_optimization/config/topic_name_define.json", 'r'))
topic_define_dict

{'纯招聘类': '纯招聘类是指以招聘职位，职位描述，薪资，待遇，要求等。也包括招聘工人。',
 '促销广告、打折信息类': '促销广告是指为了促进销售而发布的广告，通常包括打折、赠品、抽奖等促销活动信息，打折信息类是指商家为了促销而对商品或服务进行的降价优惠活动，通常以折扣比例或具体金额的方式呈现，例如“全场8折”、“满100元减20元”等。\u3000包括有明确的折扣，价格， 优惠券， 折扣，卖多少送多少，赠品，返现，礼包）',
 '大众科普，概念解释类': '大众科普是指将专有名词或者某种概念、意识传播给广大公众，它旨在让公众理解专有名词，增加概念含义的认知。该解释是能获得群众普遍认可的，不能带有个人的主观观点和思考偏向',
 '好物推荐、直播预告类': '好物推荐是实质用户根据根据使用心得推荐一些优质的产品或服务，店铺，影音书籍等，不能有明确的打折信息， 广告促销的内容，主要围绕讲使用，购买经历，晒单，薅羊毛等。直播预告类是指编辑或专家提前宣传即将进行的直播活动，包括时间、主题、嘉宾等信息，吸引观众关注并提前做好准备',
 '合租&租房广告&售房广告': '合租是指多个人共同租住一套房屋，每个人只租用其中的一部分空间，例如租用单间或者合租公寓中的一个房间。租房广告是指房东或中介发布的出租房屋的广告，通常包括房屋的位置、面积、租金、房屋设施等信息。售房广告是指房东或中介发布的出售房屋的广告，通常包括房屋的位置、面积、售价、房屋设施等信息',
 '灌水水贴': '小说，段子，故事分享，心情分享，明星打call，八卦讨论， 音乐，影视分享，心灵鸡汤等内容',
 '股市、债市动态': '大宗商品价格,期货行情波动播报,包括大盘 基金综述, 大盘,板块儿等以及个股的涨跌数字的堆积复述,不包括个人或者机构的点评分析',
 '公司公告': '通常是为了向员工、股东、客户或其他相关方面传达特定的信息或消息。公司官方公告可以涉及各种主题，例如公司业务、重要事件、财务状况、人员变动、政策变化、市场趋势',
 '公司财报': '公司财报是指公司在一定时间内的财务状况和经营业绩的报告，通常包括资产负债表、利润表和现金流量表等内容',
 '便民信息': '提供生活服务、交通、天气等实用资讯，方便日常生活。',
 '汽车经销商新闻': '汽车商家以及汽车相关的新闻报道，如新车上市、参数

# 通过prompt source 接口获取数据的embedding

In [3]:
template_code = 688
api_url = 'http://aiapi.wisers.com/openai-result-service-api/common/invoke'
semaphore_num = 80
tags = 'spam_optimization'
embedding_tools = AsyncioRequestGPT(TEMPLATE_ID=template_code, tags=tags, API_URL=api_url, semaphore_number=semaphore_num)


In [4]:
async def main(tools: AsyncioRequestGPT, data: dict, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=topic_name,
            data={
                "headline": "",
                "content": topic_defination,
            }, semaphore=semaphore
        )) for topic_name, topic_defination in data.items()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]


In [5]:
done = await main(embedding_tools, topic_define_dict)

100%|██████████| 15/15 [00:01<00:00,  8.98it/s]


In [6]:
topic_define2embedding = {dic["doc_id"]:dic['llm_response'] for dic in embedding_tools.complete_result if "doc_id" in dic and "llm_response" in dic and isinstance(dic['llm_response'], list)}

In [7]:
async def topic_name_main(tools: AsyncioRequestGPT, data: dict, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=topic_name,
            data={
                "headline": "",
                "content": topic_name,
            }, semaphore=semaphore
        )) for topic_name, topic_defination in data.items()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]

In [8]:
embedding_tools.complete_result = []
done = await topic_name_main(embedding_tools, topic_define_dict)

100%|██████████| 15/15 [00:02<00:00,  5.30it/s]


In [9]:
topic_name2embedding = {dic["doc_id"]:dic['llm_response'] for dic in embedding_tools.complete_result if "doc_id" in dic and "llm_response" in dic and isinstance(dic['llm_response'], list)}

In [10]:
pickle.dump((topic_name2embedding, topic_define2embedding), open("./config/topic2embedding.pkl", 'wb'))

## 测试生成的定义向量和标签名称向量的相似度

In [11]:
import numpy as np

arr1 = []
arr2 = []
col = []
for topic_name, _ in topic_define_dict.items():
    if topic_name in topic_name2embedding and topic_name in topic_define2embedding:
        col.append(topic_name)
        arr1.append(topic_name2embedding[topic_name])
        arr2.append(topic_define2embedding[topic_name])

        

In [12]:
pd.DataFrame(
np.around(np.dot(np.array(arr1), np.array(arr2).T), 3), columns=col, index=col).to_csv("./data/topic_vecter_similar.csv")

# 对文本内容进行向量化

In [13]:
content_data_dir = "/workspace/canton-target-sentiment/nlp_pipeline/data/datasets/sample/sequence_classification/ten_clean_data/"
file_name_list = ["train.json", 'test.json', "eval.json", "no_label_9w.json"]

In [14]:
data_list = []
for file_name in file_name_list:
    file_path = os.path.join(content_data_dir, file_name)
    part_df = pd.DataFrame(json.load(open(file_path, 'rb')))
    part_df['source'] = file_name
    data_list.append(part_df)

In [15]:
model_used_data = pd.concat(data_list)

In [16]:
async def content_embedding_main(tools: AsyncioRequestGPT, data: pd.DataFrame, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=row['docid'],
            data={
                "headline": "",
                "content": row['content'],
            }, semaphore=semaphore
        )) for index, row in data.iterrows()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]

In [17]:
embedding_tools.complete_result = []
done = await content_embedding_main(embedding_tools, model_used_data.head())

100%|██████████| 5/5 [00:01<00:00,  4.54it/s]


In [18]:
# embedding_tools.complete_result

In [19]:
model_used_data.head()

,docid,content,label,source
0,20230322A01JG83,上海电影:关于股票交易异常波动的公告 查看pdf原文公告日期:2023-03-23证券代码:...,公司公告,train.json
1,202302247733980,"平安人寿荣获 ""金理财""年度保险保障品牌奖、年度保险服务奖、年度保障型保险产品奖三项大奖 ...",待分类,train.json
2,202303161408094,"第48届中国国际""医博会""在济南市举办中工网讯(工人日报-中工网记者田国垒 通讯员杨华 葛红...",待分类,train.json
3,20230320A00HM2S,"推荐给朋友婚礼上的喜糖！因为是无蔗糖添加的,适合所有人吃 彩虹糖,好看好吃? 水果味的、不齁...",好物推荐、直播预告类,train.json
4,20230224A012HLO,"成绩斐然，看大众如何破局转型之路撰文 | 李京卓题图 | IC Photo进入电气化元年后,...",待分类,train.json


In [101]:
# 将文章内容数据按照没批50000保存文件
model_used_data['headline'] = ""
batch_number = 50000
import math
for i in range(math.ceil(model_used_data.shape[0]/batch_number)):
    start_index, end_index = i*batch_number, (i+1)*batch_number
    print(start_index, end_index)
    model_used_data[start_index:end_index].to_csv(f"./data/model_used_data{i}.csv")


0 50000
50000 100000
100000 150000
150000 200000


# 读取文章内容embedding数据

In [20]:
content_embedding_df = pd.read_csv("/workspace/spam_optimization/data/embedding/model_used_data0-688-embedding-3-small-embedding-3-small-V1-result.csv.gz")

In [21]:
# content_embedding_df.head(1).to_dict('records')
content_embedding_df['llm_response'] = content_embedding_df['llm_response'].map(lambda row: eval(row) if isinstance(row, str) else None)
nona_content_embedding_df = content_embedding_df[~content_embedding_df['llm_response'].isna()]
content_embedding_df.shape, nona_content_embedding_df.shape

((50044, 10), (49945, 10))

In [22]:
select_part_test_sample = nona_content_embedding_df.sample(frac=1).sample(1000)
select_part_test_sample.to_excel("./data/select_part_test_sample.xlsx")

# 计算文章内容和标签定义的相似度并为每个文章找到最相近的3个标签，（是否通过相似度的熵判断文章的混淆程度）

In [23]:
def cos_similar(content2embedding: dict, topic2embedding: dict):
    # content2docid = zip(content2embedding.keys(), list(range(len(content2embedding))))
    index2topic = dict(zip(list(range(len(topic2embedding))), topic2embedding.keys()))
    content_arr = np.array(list(content2embedding.values()))
    topic_arr = np.array(list(topic2embedding.values())).T
    print(content_arr.shape, topic_arr.shape)
    cos_similar_arr = np.dot(content_arr, topic_arr)
    
    cos_similar_arr_norm = (cos_similar_arr-np.min(cos_similar_arr))/(np.max(cos_similar_arr)-np.min(cos_similar_arr))

    sorted_arr = np.argsort(cos_similar_arr, axis=1)
    sorted_cos_similar_arr = np.sort(cos_similar_arr, axis=1)
    # print(sorted_arr[0,:], "\n", cos_similar_arr[0, :], "\n", sorted_cos_similar_arr[0, :])

    
    content_most_similar_df = pd.DataFrame(sorted_arr[:, -5:], columns=['top1', 'top2', 'top3', "top4", "top5"])
    content_most_similar_df['docid'] = list(content2embedding.keys())
    content_most_similar_df['top1_name'] = content_most_similar_df['top1'].map(index2topic)
    content_most_similar_df['top1_prob'] = sorted_cos_similar_arr[:, -1]
    content_most_similar_df['top2_name'] = content_most_similar_df['top2'].map(index2topic)
    content_most_similar_df['top2_prob'] = sorted_cos_similar_arr[:, -2]
    content_most_similar_df['top3_name'] = content_most_similar_df['top3'].map(index2topic)
    content_most_similar_df['top3_prob'] = sorted_cos_similar_arr[:, -3]
    
    content_most_similar_df['top4_name'] = content_most_similar_df['top4'].map(index2topic)
    content_most_similar_df['top4_prob'] = sorted_cos_similar_arr[:, -3]
    content_most_similar_df['top5_name'] = content_most_similar_df['top5'].map(index2topic)
    content_most_similar_df['top5_prob'] = sorted_cos_similar_arr[:, -3]
    drop_cols = ["top1", "top2", "top3"]
    return content_most_similar_df.drop(columns=drop_cols)
    


In [24]:
content2embedding = {dic["docid"]:dic['llm_response'] for index, dic in select_part_test_sample.iterrows() if "docid" in dic and "llm_response" in dic and isinstance(dic['llm_response'], list)}
content2embedding.__len__()

996

In [25]:
content_and_name_most_similar_df = cos_similar(content2embedding, topic_name2embedding)

(996, 1536) (1536, 15)


In [26]:
content_and_define_most_similar_df = cos_similar(content2embedding, topic_define2embedding)

(996, 1536) (1536, 15)


In [27]:
content_and_define_most_similar_df.head()

,top4,top5,docid,top1_name,top1_prob,top2_name,top2_prob,top3_name,top3_prob,top4_name,top4_prob,top5_name,top5_prob
0,0,10,202303147144401,提问求答鉴定类,0.326589,公司财报,0.300008,灌水水贴,0.297395,股市、债市动态,0.297395,公司公告,0.297395
1,4,10,202303091396210,汽车经销商新闻,0.292806,提问求答鉴定类,0.253772,股市、债市动态,0.249672,公司财报,0.249672,公司公告,0.249672
2,8,2,20230227A0261B7,好物推荐、直播预告类,0.322935,公司公告,0.276871,纯招聘类,0.258700,便民信息,0.258700,维修服务或客服热线,0.258700
3,10,4,20230320A00HAKV,好物推荐、直播预告类,0.350727,汽车经销商新闻,0.329309,股市、债市动态,0.294523,公司公告,0.294523,公司财报,0.294523
4,4,0,202302275419308,维修服务或客服热线,0.277153,灌水水贴,0.224435,公司公告,0.214941,公司财报,0.214941,股市、债市动态,0.214941


In [28]:
content_and_name_most_similar_df['docid'] = content_and_name_most_similar_df['docid'].astype(str)
content_and_define_most_similar_df['docid'] = content_and_define_most_similar_df['docid'].astype(str)

similar_topic_df = select_part_test_sample.merge(content_and_name_most_similar_df, on='docid', how='left').merge(content_and_define_most_similar_df, on='docid', how='left')
similar_topic_df.head()

,Unnamed: 0,docid,content,label,source,headline,llm_task_id,llm_status,llm_response,llm_msg,...,top1_name_y,top1_prob_y,top2_name_y,top2_prob_y,top3_name_y,top3_prob_y,top4_name_y,top4_prob_y,top5_name_y,top5_prob_y
0,40120,202303147144401,"山东财经大学校友经济发展论坛(2023)成功举办为""汇集校友 聚创泉城""，发挥""财大系校友""...",待分类,train.json,NaN,model_used_data0.csv_34449c36-f4bc-3252-b4b3-f...,0.0,"[0.01423714030534029, -0.053102362900972366, 0...",NaN,...,提问求答鉴定类,0.326589,公司财报,0.300008,灌水水贴,0.297395,股市、债市动态,0.297395,公司公告,0.297395
1,25244,202303091396210,"安永内部对税务部门命运意见不一,据悉已暂停分拆咨询业务计划据英国《金融时报》和《华尔街日报》...",待分类,train.json,NaN,model_used_data0.csv_b9861acc-ce98-3bdc-8242-3...,0.0,"[0.023206759244203568, 0.02385387010872364, 0....",NaN,...,汽车经销商新闻,0.292806,提问求答鉴定类,0.253772,股市、债市动态,0.249672,公司财报,0.249672,公司公告,0.249672
2,16714,20230227A0261B7,华电宿州公司：狠抓技能培训 助力青工成长为进一步提高青年职工的业务素质和技能水平，华电宿州公...,待分类,train.json,NaN,model_used_data0.csv_c3d0a88e-72d1-3944-aec5-b...,0.0,"[-0.006939513608813286, 0.005614547058939934, ...",NaN,...,好物推荐、直播预告类,0.322935,公司公告,0.276871,纯招聘类,0.258700,便民信息,0.258700,维修服务或客服热线,0.258700
3,7008,20230320A00HAKV,华翔股份董秘回复：公司所参与建设的光伏发电项目目前正在积极地进行筹建华翔股份(603112)...,公司财报,train.json,NaN,model_used_data0.csv_81e186f9-d659-3aea-b83d-9...,0.0,"[0.0057653384283185005, -0.01999935507774353, ...",NaN,...,好物推荐、直播预告类,0.350727,汽车经销商新闻,0.329309,股市、债市动态,0.294523,公司公告,0.294523,公司财报,0.294523
4,21380,202302275419308,香港中文大学(深圳)发布2022届本科毕业生就业质量报告，平均年薪16.69万元 2月26...,待分类,train.json,NaN,model_used_data0.csv_143010d2-17b0-3e6b-9de0-5...,0.0,"[0.005017169285565615, 0.024495039135217667, 0...",NaN,...,维修服务或客服热线,0.277153,灌水水贴,0.224435,公司公告,0.214941,公司财报,0.214941,股市、债市动态,0.214941


# 通过prompt验证标签名称和定义召回的类别是否正确

In [248]:
topic_cols = [cols for cols in similar_topic_df.columns if 'top' in cols and "name" in cols]
origin_cols = ['docid', 'content', 'label', "source"]
# 将两个top5的召回标签和定义合并在一起
def generate_possible_topic_info(row, topic_cols, topic_define_dict):
    possible_topic_info = {}
    for col in topic_cols:
        if row[col] == "股市、债市动态": 
            possible_topic_info["股市、债市动态"] = topic_define_dict["股市债市动态"]  
        else: 
            possible_topic_info[row[col]] = topic_define_dict[row[col]]
    return possible_topic_info

['top1_name_x', 'top2_name_x', 'top3_name_x', 'top4_name_x', 'top5_name_x', 'top1_name_y', 'top2_name_y', 'top3_name_y', 'top4_name_y', 'top5_name_y']


In [249]:
prompt_test_df = similar_topic_df[similar_topic_df['label']!='待分类'].sample(100, random_state=1)

In [250]:
prompt_test_df['sign5'] = prompt_test_df.apply(lambda row: row['label'] in [row[cols] for cols in topic_cols], axis=1)
topic4_cols = ['top1_name_x', 'top2_name_x', 'top3_name_x', 'top4_name_x', 'top1_name_y', 'top2_name_y', 'top3_name_y', 'top4_name_y']
prompt_test_df['sign4'] = prompt_test_df.apply(lambda row: row['label'] in [row[cols] for cols in topic4_cols], axis=1)
topic3_cols = ['top1_name_x', 'top2_name_x', 'top3_name_x', 'top1_name_y', 'top2_name_y', 'top3_name_y']
prompt_test_df['sign3'] = prompt_test_df.apply(lambda row: row['label'] in [row[cols] for cols in topic3_cols], axis=1)



In [251]:
prompt_test_df[prompt_test_df['sign5']==True].shape, prompt_test_df[prompt_test_df['sign4']==True].shape, prompt_test_df[prompt_test_df['sign3']==True].shape

((91, 37), (56, 37), (36, 37))

In [252]:
prompt_test_df['docid'] = prompt_test_df['docid'].astype(str)


In [253]:
define_col = []
for col in topic_cols:
    
    new_col = col.replace("_name_", "_define_")
    define_col.append(new_col)
    prompt_test_df[new_col] = prompt_test_df[col].map(topic_define_dict)
    
    

In [254]:
prompt_test_df.head(1)

,Unnamed: 0,docid,content,label,source,headline,llm_task_id,llm_status,llm_response,llm_msg,...,top1_define_x,top2_define_x,top3_define_x,top4_define_x,top5_define_x,top1_define_y,top2_define_y,top3_define_y,top4_define_y,top5_define_y
155,13937,202303212710287,#时光里幸福一帧#@苏宁易购@海信电视@vivo#时光里幸福一帧#@苏宁易购@海信电视@vi...,促销广告、打折信息类,train.json,NaN,model_used_data0.csv_652fd58f-0089-3a9a-a933-6...,0.0,"[0.06530198454856873, -0.043340038508176804, -...",NaN,...,通过不同渠道进行跨国购物，例如海淘、带货、代购,通常是为了向员工、股东、客户或其他相关方面传达特定的信息或消息。公司官方公告可以涉及各种主题...,小说，段子，故事分享，心情分享，明星打call，八卦讨论， 音乐，影视分享，心灵鸡汤等内容,好物推荐是实质用户根据根据使用心得推荐一些优质的产品或服务，店铺，影音书籍等，不能有明确的打...,促销广告是指为了促进销售而发布的广告，通常包括打折、赠品、抽奖等促销活动信息，打折信息类是指...,发表提问，求专业知识解答与物品真伪鉴定，互动交流解疑惑,提供生活服务、交通、天气等实用资讯，方便日常生活。,小说，段子，故事分享，心情分享，明星打call，八卦讨论， 音乐，影视分享，心灵鸡汤等内容,促销广告是指为了促进销售而发布的广告，通常包括打折、赠品、抽奖等促销活动信息，打折信息类是指...,好物推荐是实质用户根据根据使用心得推荐一些优质的产品或服务，店铺，影音书籍等，不能有明确的打...


In [255]:


prompt_test_df['possible_topic_info'] = prompt_test_df.apply(generate_possible_topic_info, args=(topic_cols, topic_define_dict,), axis=1)

In [257]:
prompt_test_df[origin_cols+topic_cols+define_col+['sign3','sign4','sign5', "possible_topic_info"]].to_csv("./data/spam_prompt_test_df.csv")

In [260]:
prompt_test_df['len'] = prompt_test_df['possible_topic_info'].map(len)


In [261]:
prompt_test_df['len'].value_counts()

7    38
6    28
8    23
9     7
5     4
Name: len, dtype: int64

In [263]:
pd.Series(topic_define_dict).to_csv("topic_define.csv")

# 从有标签的数据中采样500条数据，并将金融行业数据选取500条

In [29]:
def group_sample(data: pd.DataFrame, sample_number=50):
    sample_list = []
    for label, groupdf in data.groupby("label"):
        if groupdf.shape[0] >= sample_number:
            sample_list.append(groupdf.sample(sample_number))
        else:
            sample_list.append(groupdf)

    return pd.concat(sample_list)

have_label_test_sample = group_sample(content_embedding_df)

In [30]:
have_label_test_sample.head()

,Unnamed: 0,docid,content,label,source,headline,llm_task_id,llm_status,llm_response,llm_msg
44902,1368,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,NaN,model_used_data0.csv_cb10d15c-ae79-3dec-aab7-2...,0.0,"[-0.008920187130570412, 0.0063757398165762424,...",NaN
19852,19832,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,NaN,model_used_data0.csv_f835d151-3477-3b51-87ad-2...,0.0,"[-0.008494753390550613, -0.03978944942355156, ...",NaN
20168,20148,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,NaN,model_used_data0.csv_7b612578-2b21-3e9b-a4bb-c...,0.0,"[0.020520830526947975, 0.028596319258213043, -...",NaN
15307,15291,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,NaN,model_used_data0.csv_df773f60-22ff-35db-94ff-c...,0.0,"[0.005606198217719793, 0.019215792417526245, 0...",NaN
46613,3079,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,NaN,model_used_data0.csv_c2cef771-fe1b-393a-999e-b...,0.0,"[0.02227720245718956, 0.010503695346415043, 0....",NaN


In [31]:
five_client_test_sample = pd.read_json("/workspace/canton-target-sentiment/nlp_pipeline/data/datasets/sample/sequence_classification/nine_clean_data/five_client_manual_fix_0829.json")

In [32]:
financial_test_sample = pd.read_excel("/workspace/spam_optimization/data/废文分类汇总2024-06-04 (1).xlsx", sheet_name='金融行业废文')
financial_test_sample['content'] = financial_test_sample.apply(lambda row: row['内文'] if str(row['标题']) in str(row['内文']) else str(row['标题'])+str(row['内文']), axis=1)
financial_test_sample = financial_test_sample[(~financial_test_sample['content'].isna()) & (financial_test_sample['content']!="")].rename(columns={"文章编号": "docid"})
financial_test_sample['label'] = financial_test_sample['内容标签']

In [36]:
five_client_test_sample['source'] = "five_client_test_sample"
financial_test_sample['source'] = 'financial_test_sample'
business_test_sample = pd.concat([five_client_test_sample[['docid', 'content', 'label', 'source']], financial_test_sample[['docid', 'content', 'label', 'source']]]).drop_duplicates(subset=['docid'])
business_test_sample_500 = business_test_sample.sample(500, random_state=1)

In [37]:
embedding_tools.complete_result = []
done = await content_embedding_main(embedding_tools, business_test_sample_500)

 99%|█████████▉| 496/500 [00:04<00:00, 23.66it/s]2024-06-20 17:05:59.988 | ERROR    | asyncio_gpt:request_post:41 - response result is except， <ClientResponse(http://aiapi.wisers.com/openai-result-service-api/common/invoke) [200 OK]>
<CIMultiDictProxy('date': 'Thu, 20 Jun 2024 09:05:56 GMT', 'server': 'envoy', 'content-length': '474', 'content-type': 'application/json', 'x-envoy-upstream-service-time': '2733')>

2024-06-20 17:06:00.020 | ERROR    | asyncio_gpt:request_post:41 - response result is except， <ClientResponse(http://aiapi.wisers.com/openai-result-service-api/common/invoke) [200 OK]>
<CIMultiDictProxy('date': 'Thu, 20 Jun 2024 09:05:56 GMT', 'server': 'envoy', 'content-length': '476', 'content-type': 'application/json', 'x-envoy-upstream-service-time': '2445')>

100%|██████████| 500/500 [00:05<00:00, 90.72it/s]


In [38]:
docid2embedding = {dic['doc_id']: dic['llm_response'] for dic in embedding_tools.complete_result if "doc_id" in dic and "llm_response" in dic}
docid2embedding.__len__()

498

In [39]:
business_test_sample_500['llm_response'] = business_test_sample_500['docid'].map(docid2embedding)


In [40]:
have_label_test_sample.columns

Index(['Unnamed: 0', 'docid', 'content', 'label', 'source', 'headline',
       'llm_task_id', 'llm_status', 'llm_response', 'llm_msg'],
      dtype='object')

In [42]:
complete_test_df = pd.concat([have_label_test_sample[['docid', 'content', 'label', 'source', 'llm_response']], business_test_sample_500[['docid', 'content', 'label', 'source', 'llm_response']]])
complete_test_df.shape

(1000, 5)

In [44]:

complete_test_docid2embedding = {row['docid']:row['llm_response'] for index, row in complete_test_df[~complete_test_df['llm_response'].isna()].iterrows()}

content_and_name_similar = cos_similar(complete_test_docid2embedding, topic_name2embedding)
content_and_define_similar = cos_similar(complete_test_docid2embedding, topic_define2embedding)

(994, 1536) (1536, 15)
(994, 1536) (1536, 15)


In [45]:
content_and_name_similar.head()

,top4,top5,docid,top1_name,top1_prob,top2_name,top2_prob,top3_name,top3_prob,top4_name,top4_prob,top5_name,top5_prob
0,5,10,202205262392895,合租&租房广告&售房广告,0.290578,维修服务或客服热线,0.286016,假冒仿制商品,0.253500,跨洋购物,0.253500,好物推荐、直播预告类,0.253500
1,7,10,c069cb10d2e60a8e6aee82d5638803f6,跨洋购物,0.348942,促销广告、打折信息类,0.313595,灌水水贴,0.310440,假冒仿制商品,0.310440,好物推荐、直播预告类,0.310440
2,10,3,202303206118888,合租&租房广告&售房广告,0.336226,公司公告,0.282415,跨洋购物,0.278873,好物推荐、直播预告类,0.278873,促销广告、打折信息类,0.278873
3,0,5,20230323A05KI1E,公司公告,0.330342,汽车经销商新闻,0.327259,合租&租房广告&售房广告,0.302659,灌水水贴,0.302659,跨洋购物,0.302659
4,7,1,20230322A01G7Y1,公司财报,0.345675,跨洋购物,0.331557,促销广告、打折信息类,0.327215,假冒仿制商品,0.327215,汽车经销商新闻,0.327215


In [46]:
content_and_define_similar.head()

,top4,top5,docid,top1_name,top1_prob,top2_name,top2_prob,top3_name,top3_prob,top4_name,top4_prob,top5_name,top5_prob
0,12,7,202205262392895,便民信息,0.285306,灌水水贴,0.277640,好物推荐、直播预告类,0.260867,提问求答鉴定类,0.260867,跨洋购物,0.260867
1,7,5,c069cb10d2e60a8e6aee82d5638803f6,假冒仿制商品,0.365233,促销广告、打折信息类,0.338138,股市、债市动态,0.324761,跨洋购物,0.324761,好物推荐、直播预告类,0.324761
2,13,14,202303206118888,提问求答鉴定类,0.372233,便民信息,0.343823,好物推荐、直播预告类,0.339719,促销广告、打折信息类,0.339719,灌水水贴,0.339719
3,5,7,20230323A05KI1E,促销广告、打折信息类,0.287441,提问求答鉴定类,0.233171,汽车经销商新闻,0.231642,好物推荐、直播预告类,0.231642,跨洋购物,0.231642
4,7,0,20230322A01G7Y1,好物推荐、直播预告类,0.366286,假冒仿制商品,0.358491,促销广告、打折信息类,0.335790,跨洋购物,0.335790,股市、债市动态,0.335790


# 将两路召回结果合并在一起

In [50]:
content_and_name_similar['docid'] = content_and_name_similar['docid'].astype(str)
content_and_define_similar['docid'] = content_and_define_similar['docid'].astype(str)

complete_recall_topic_df = complete_test_df.merge(content_and_define_similar, on='docid', how='left').merge(content_and_name_similar, on='docid', how='left').dropna()
complete_recall_topic_df.head()

,docid,content,label,source,llm_response,top4_x,top5_x,top1_name_x,top1_prob_x,top2_name_x,...,top1_name_y,top1_prob_y,top2_name_y,top2_prob_y,top3_name_y,top3_prob_y,top4_name_y,top4_prob_y,top5_name_y,top5_prob_y
0,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,"[-0.008920187130570412, 0.0063757398165762424,...",12.0,7.0,便民信息,0.285306,灌水水贴,...,合租&租房广告&售房广告,0.290578,维修服务或客服热线,0.286016,假冒仿制商品,0.253500,跨洋购物,0.253500,好物推荐、直播预告类,0.253500
1,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,"[-0.008494753390550613, -0.03978944942355156, ...",7.0,5.0,假冒仿制商品,0.365233,促销广告、打折信息类,...,跨洋购物,0.348942,促销广告、打折信息类,0.313595,灌水水贴,0.310440,假冒仿制商品,0.310440,好物推荐、直播预告类,0.310440
2,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,"[0.020520830526947975, 0.028596319258213043, -...",13.0,14.0,提问求答鉴定类,0.372233,便民信息,...,合租&租房广告&售房广告,0.336226,公司公告,0.282415,跨洋购物,0.278873,好物推荐、直播预告类,0.278873,促销广告、打折信息类,0.278873
3,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,"[0.005606198217719793, 0.019215792417526245, 0...",5.0,7.0,促销广告、打折信息类,0.287441,提问求答鉴定类,...,公司公告,0.330342,汽车经销商新闻,0.327259,合租&租房广告&售房广告,0.302659,灌水水贴,0.302659,跨洋购物,0.302659
4,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,"[0.02227720245718956, 0.010503695346415043, 0....",7.0,0.0,好物推荐、直播预告类,0.366286,假冒仿制商品,...,公司财报,0.345675,跨洋购物,0.331557,促销广告、打折信息类,0.327215,假冒仿制商品,0.327215,汽车经销商新闻,0.327215


# 获取召回标签的定义，通过召回标签prompt进行选择打标

## 组合prompt输入参数

In [53]:
complete_recall_topic_df.to_pickle("./data/complete_recall_topic_df.pkl")

In [56]:
topic_cols = [cols for cols in similar_topic_df.columns if 'top' in cols and "name" in cols]
origin_cols = ['docid', 'content', 'label', "source"]
# 将两个top5的召回标签和定义合并在一起
def generate_possible_topic_info(row, topic_cols, topic_define_dict):
    possible_topic_info = {}
    for col in topic_cols:
        possible_topic_info[row[col]] = topic_define_dict[row[col]]

        # if row[col] == "股市、债市动态": 
        #     possible_topic_info["股市、债市动态"] = topic_define_dict["股市债市动态"]  
        # else: 
        #     possible_topic_info[row[col]] = topic_define_dict[row[col]]
    return possible_topic_info

In [60]:
complete_recall_topic_df['possible_topic_info'] = complete_recall_topic_df.apply(generate_possible_topic_info, args=(topic_cols, topic_define_dict, ), axis=1)


In [61]:
complete_recall_topic_df[origin_cols+topic_cols+['possible_topic_info']].head()

,docid,content,label,source,top1_name_x,top2_name_x,top3_name_x,top4_name_x,top5_name_x,top1_name_y,top2_name_y,top3_name_y,top4_name_y,top5_name_y,possible_topic_info
0,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,便民信息,灌水水贴,好物推荐、直播预告类,提问求答鉴定类,跨洋购物,合租&租房广告&售房广告,维修服务或客服热线,假冒仿制商品,跨洋购物,好物推荐、直播预告类,"{'便民信息': '提供生活服务、交通、天气等实用资讯，方便日常生活。', '灌水水贴': ..."
1,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,假冒仿制商品,促销广告、打折信息类,股市、债市动态,跨洋购物,好物推荐、直播预告类,跨洋购物,促销广告、打折信息类,灌水水贴,假冒仿制商品,好物推荐、直播预告类,{'假冒仿制商品': '在设计、外观或功能上模仿或参照知名品牌、高端产品，但在品牌标识、材料...
2,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,提问求答鉴定类,便民信息,好物推荐、直播预告类,促销广告、打折信息类,灌水水贴,合租&租房广告&售房广告,公司公告,跨洋购物,好物推荐、直播预告类,促销广告、打折信息类,"{'提问求答鉴定类': '发表提问，求专业知识解答与物品真伪鉴定，互动交流解疑惑', '便民..."
3,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,促销广告、打折信息类,提问求答鉴定类,汽车经销商新闻,好物推荐、直播预告类,跨洋购物,公司公告,汽车经销商新闻,合租&租房广告&售房广告,灌水水贴,跨洋购物,{'促销广告、打折信息类': '促销广告是指为了促进销售而发布的广告，通常包括打折、赠品、抽...
4,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,好物推荐、直播预告类,假冒仿制商品,促销广告、打折信息类,跨洋购物,股市、债市动态,公司财报,跨洋购物,促销广告、打折信息类,假冒仿制商品,汽车经销商新闻,{'好物推荐、直播预告类': '好物推荐是实质用户根据根据使用心得推荐一些优质的产品或服务，...


## 第一步prompt从召回的多标签中选择最合理的标签

In [115]:
template_code = 966
api_url = 'http://aiapi.wisers.com/openai-result-service-api/common/invoke'
semaphore_num = 100
tags = 'spam_optimization'
multi_label_tools = AsyncioRequestGPT(TEMPLATE_ID=template_code, tags=tags, API_URL=api_url, semaphore_number=semaphore_num)


In [116]:
async def multi_label_select(tools: AsyncioRequestGPT, data: dict, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=row['docid'],
            data={
                "possible_topic_info": row["possible_topic_info"],
                "content": row['content'],
            }, semaphore=semaphore
        )) for index, row in data.iterrows()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]

In [119]:
multi_label_tools.complete_result = []
multi_label_select_done = await multi_label_select(multi_label_tools, complete_recall_topic_df, semaphore_num)

  0%|          | 0/965 [00:00<?, ?it/s]/opt/conda/lib/python3.8/asyncio/tasks.py:907: RuntimeWarning: coroutine 'multi_label_evaluation' was never awaited
  outer.add_done_callback(_outer_done_callback)
 38%|███▊      | 367/965 [00:33<00:48, 12.29it/s]2024-06-21 11:39:48.183 | ERROR    | asyncio_gpt:request_post:41 - response result is except， <ClientResponse(http://aiapi.wisers.com/openai-result-service-api/common/invoke) [200 OK]>
<CIMultiDictProxy('date': 'Fri, 21 Jun 2024 03:39:45 GMT', 'server': 'envoy', 'content-length': '212', 'content-type': 'application/json', 'x-envoy-upstream-service-time': '2408')>

 69%|██████▉   | 666/965 [01:14<00:22, 13.31it/s]2024-06-21 11:40:29.547 | ERROR    | asyncio_gpt:request_post:41 - response result is except， <ClientResponse(http://aiapi.wisers.com/openai-result-service-api/common/invoke) [200 OK]>
<CIMultiDictProxy('date': 'Fri, 21 Jun 2024 03:40:11 GMT', 'server': 'envoy', 'content-length': '2623', 'content-type': 'application/json', 'x-envo

In [127]:
multi_label_tools.complete_result[0]

{'doc_id': '202303206118888',
 'llm_response': {'result': [{'reasonable_topic': '促销广告、打折信息类',
    'topic_relate_original': "春日见喜，镇江月星'狂享家'活动圆满收官！3月19日 镇江月星'狂享家'活动 圆满收官! 三月,春日的气息越来越浓 家装的热情也不断高涨起来 下面带大家感受一下 月星'狂享家'的现场盛况~ 无需购物抽大奖、居家壕礼满额赠 家装大课堂等精彩活动环节 吸引了众多顾客前来参与 活动现场热闹非凡! 无论是活动区还是收银台处 都排起了长队,顾客络绎不绝 各楼层展位也强势出击 巨惠不停、签单不断! 01 无需购物抽奖 抽雅迪电动车 无需购物即可抽奖 百分百中奖,百分百惊喜 更有运气爆棚的顾客 一抽就是电动车,手气炸裂! 02 居家壕礼满额赠 60台洗地机全城乐享 洗地机、咖啡机 电动牙刷、煤油造型台灯 超值居家壕礼拿到手软 引发一波又一波的开单热潮! 03 0511家装大课堂 轻松扮靓理想家 干货满满的家装课堂 受到了现场顾客的一致好评! 而互动秒杀环节 更将现场气氛推向了高潮 顾客积极参与互动 空气炸锅、电炒蒸煮一体锅 喜临门枕头等丰厚的奖品 让每位顾客都满载而",
    'confidence': '高'}]}}

In [131]:
content_multi_label_res = [(
        multi_label_result['doc_id'], 
        dic['reasonable_topic'] if "reasonable_topic" in dic else None, 
        dic['topic_relate_original'] if "topic_relate_original" in dic else None, 
        dic['confidence'] if "confidence" in dic else None
    )
    for multi_label_result in multi_label_tools.complete_result  for dic in multi_label_result['llm_response']['result'] if 'doc_id' in multi_label_result and "llm_response" in multi_label_result and "result" in multi_label_result['llm_response']]

content_multi_label_res.__len__()


964

In [133]:
content_multi_label_res_df = pd.DataFrame(content_multi_label_res, columns=['docid', 'reasonable_topic', 'topic_relate_original', 'confidence'])
content_multi_label_res_df.head()

,docid,reasonable_topic,topic_relate_original,confidence
0,202303206118888,促销广告、打折信息类,春日见喜，镇江月星'狂享家'活动圆满收官！3月19日 镇江月星'狂享家'活动 圆满收官! 三...,高
1,20230323A05KI1E,促销广告、打折信息类,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,高
2,202205262392895,跨洋购物,美国包邮:北美热销厨具!中国包邮:时装鞋帽装饰品、美容美发品、健康养生品!,中
3,20230322A01G7Y1,其他,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,中
4,c069cb10d2e60a8e6aee82d5638803f6,促销广告、打折信息类,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",高


In [136]:
complete_recall_topic_df['embedding_recall_topics'] = complete_recall_topic_df['possible_topic_info'].map(lambda row: list(row.keys()) if isinstance(row, dict) else None)
need_save_cols = ['docid', 'content', 'label', 'source', 'embedding_recall_topics']
complete_recall_topic_add_gpt_topic_df = complete_recall_topic_df[need_save_cols].merge(content_multi_label_res_df, on='docid', how='left')
complete_recall_topic_add_gpt_topic_df.head()

,docid,content,label,source,embedding_recall_topics,reasonable_topic,topic_relate_original,confidence
0,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,"[便民信息, 灌水水贴, 好物推荐、直播预告类, 提问求答鉴定类, 跨洋购物, 合租&租房广...",跨洋购物,美国包邮:北美热销厨具!中国包邮:时装鞋帽装饰品、美容美发品、健康养生品!,中
1,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,"[假冒仿制商品, 促销广告、打折信息类, 股市、债市动态, 跨洋购物, 好物推荐、直播预告类...",促销广告、打折信息类,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",高
2,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,"[提问求答鉴定类, 便民信息, 好物推荐、直播预告类, 促销广告、打折信息类, 灌水水贴, ...",促销广告、打折信息类,春日见喜，镇江月星'狂享家'活动圆满收官！3月19日 镇江月星'狂享家'活动 圆满收官! 三...,高
3,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,"[促销广告、打折信息类, 提问求答鉴定类, 汽车经销商新闻, 好物推荐、直播预告类, 跨洋购...",促销广告、打折信息类,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,高
4,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,"[好物推荐、直播预告类, 假冒仿制商品, 促销广告、打折信息类, 跨洋购物, 股市、债市动态...",其他,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,中


In [141]:
complete_recall_topic_add_gpt_topic_df['second_confirmation_topic'] = complete_recall_topic_add_gpt_topic_df['reasonable_topic'].map(lambda label: {
        "topic_name": label,
        "topic_define": topic_define_dict[label]
    } if label in topic_define_dict else None)

In [142]:
complete_recall_topic_add_gpt_topic_df.head()

,docid,content,label,source,embedding_recall_topics,reasonable_topic,topic_relate_original,confidence,second_confirmation_topic
0,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,"[便民信息, 灌水水贴, 好物推荐、直播预告类, 提问求答鉴定类, 跨洋购物, 合租&租房广...",跨洋购物,美国包邮:北美热销厨具!中国包邮:时装鞋帽装饰品、美容美发品、健康养生品!,中,"{'topic_name': '跨洋购物', 'topic_define': '通过不同渠道..."
1,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,"[假冒仿制商品, 促销广告、打折信息类, 股市、债市动态, 跨洋购物, 好物推荐、直播预告类...",促销广告、打折信息类,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",高,"{'topic_name': '促销广告、打折信息类', 'topic_define': '..."
2,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,"[提问求答鉴定类, 便民信息, 好物推荐、直播预告类, 促销广告、打折信息类, 灌水水贴, ...",促销广告、打折信息类,春日见喜，镇江月星'狂享家'活动圆满收官！3月19日 镇江月星'狂享家'活动 圆满收官! 三...,高,"{'topic_name': '促销广告、打折信息类', 'topic_define': '..."
3,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,"[促销广告、打折信息类, 提问求答鉴定类, 汽车经销商新闻, 好物推荐、直播预告类, 跨洋购...",促销广告、打折信息类,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,高,"{'topic_name': '促销广告、打折信息类', 'topic_define': '..."
4,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,"[好物推荐、直播预告类, 假冒仿制商品, 促销广告、打折信息类, 跨洋购物, 股市、债市动态...",其他,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,中,None


In [172]:
complete_recall_topic_add_gpt_topic_df['reasonable_topic'].value_counts()

公司公告            243
股市、债市动态         148
汽车经销商新闻         117
公司财报             75
好物推荐、直播预告类       58
纯招聘类             47
便民信息             47
促销广告、打折信息类       46
合租&租房广告&售房广告     45
大众科普，概念解释类       45
灌水水贴             44
其他               20
维修服务或客服热线        12
售房广告              8
假冒仿制商品            5
跨洋购物              4
提问求答鉴定类           2
公司合作公告            1
能源化工简报            1
创新营销赋能            1
政府信息              1
Name: reasonable_topic, dtype: int64

## 第二步prompt验证第一步中打得标签是否正确

In [171]:
template_code = 968
api_url = 'http://aiapi.wisers.com/openai-result-service-api/common/invoke'
semaphore_num = 100
tags = 'spam_optimization'
label_evaluation_tools = AsyncioRequestGPT(TEMPLATE_ID=template_code, tags=tags, API_URL=api_url, semaphore_number=semaphore_num)


In [173]:
async def label_evaluation(tools: AsyncioRequestGPT, data: dict, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=row['docid'],
            data={
                # "possible_topic_info": row["second_confirmation_topic"],
                "content": row['content'],
                "label": row['reasonable_topic'],
                "possible_topic_info": topic_define_dict[row['reasonable_topic']] if row['reasonable_topic'] in topic_define_dict else None
            }, semaphore=semaphore
        )) for index, row in data.iterrows()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]

In [176]:
label_evaluation_tools.complete_result = []
label_evaluation_done = await label_evaluation(label_evaluation_tools, complete_recall_topic_add_gpt_topic_df[~complete_recall_topic_add_gpt_topic_df['second_confirmation_topic'].isna()], semaphore_num)

100%|██████████| 938/938 [01:07<00:00, 13.86it/s]


In [177]:
label_evaluation_tools.complete_result.__len__()

938

In [178]:
# for result in label_evaluation_tools.complete_result:
#     if "doc_id" in result and "llm_response" in result and "topic_is_right" in result['llm_response'] and 'explanation' in result['llm_response']:
        

label_evaluation_result = pd.DataFrame([(result['doc_id'], result['llm_response']['topic_name'], result['llm_response']['topic_is_right'], result['llm_response']['explanation']) for result in label_evaluation_tools.complete_result 
if "doc_id" in result and "llm_response" in result and isinstance(result['llm_response'], dict) and "topic_is_right" in result['llm_response'] and 'explanation' in result['llm_response']], 
                                       columns=['docid', "topic_name", 'topic_is_right', "explanation"])

In [179]:
label_evaluation_result.shape

(937, 4)

In [180]:
label_evaluation_result['docid'] = label_evaluation_result['docid'].astype(str)
two_step_tag_result = complete_recall_topic_add_gpt_topic_df.merge(label_evaluation_result, on='docid', how='left')
two_step_tag_result.shape

(1051, 12)

In [181]:
two_step_tag_result.columns

Index(['docid', 'content', 'label', 'source', 'embedding_recall_topics',
       'reasonable_topic', 'topic_relate_original', 'confidence',
       'second_confirmation_topic', 'topic_name', 'topic_is_right',
       'explanation'],
      dtype='object')

In [182]:
two_step_tag_result['docid'] = two_step_tag_result['docid'].astype(str)
two_step_tag_result['sign'] = two_step_tag_result.apply(lambda row: row['label'] in row['embedding_recall_topics'], axis=1)
two_step_tag_result.to_csv("./data/complete_step_tag_result.csv")

# 直接将embedding召回的结果进行多标签确认，选择GPT任务正确的类型

In [103]:
template_code = 964
api_url = 'http://aiapi.wisers.com/openai-result-service-api/common/invoke'
semaphore_num = 100
tags = 'spam_optimization'
multi_label_evaluation_tools = AsyncioRequestGPT(TEMPLATE_ID=template_code, tags=tags, API_URL=api_url, semaphore_number=semaphore_num)


In [104]:
complete_recall_topic_df[origin_cols+topic_cols+['possible_topic_info']].head()

,docid,content,label,source,top1_name_x,top2_name_x,top3_name_x,top4_name_x,top5_name_x,top1_name_y,top2_name_y,top3_name_y,top4_name_y,top5_name_y,possible_topic_info
0,202205262392895,​西雅图/加拿大周边游:海岸星光号列车+波特兰+101公路神奇海岸线+神木步道+红木公园+旧...,促销广告、打折信息类,test.json,便民信息,灌水水贴,好物推荐、直播预告类,提问求答鉴定类,跨洋购物,合租&租房广告&售房广告,维修服务或客服热线,假冒仿制商品,跨洋购物,好物推荐、直播预告类,"{'便民信息': '提供生活服务、交通、天气等实用资讯，方便日常生活。', '灌水水贴': ..."
1,c069cb10d2e60a8e6aee82d5638803f6,"【平论+赞随机揪免单】卷后42.9【百草味】罐装夏威夷果奶油味500g,坚果旺季来了,开壳不...",促销广告、打折信息类,train.json,假冒仿制商品,促销广告、打折信息类,股市、债市动态,跨洋购物,好物推荐、直播预告类,跨洋购物,促销广告、打折信息类,灌水水贴,假冒仿制商品,好物推荐、直播预告类,{'假冒仿制商品': '在设计、外观或功能上模仿或参照知名品牌、高端产品，但在品牌标识、材料...
2,202303206118888,"春日见喜，镇江月星""狂享家""活动圆满收官！3月19日 镇江月星""狂享家""活动 圆满收官! 三...",促销广告、打折信息类,train.json,提问求答鉴定类,便民信息,好物推荐、直播预告类,促销广告、打折信息类,灌水水贴,合租&租房广告&售房广告,公司公告,跨洋购物,好物推荐、直播预告类,促销广告、打折信息类,"{'提问求答鉴定类': '发表提问，求专业知识解答与物品真伪鉴定，互动交流解疑惑', '便民..."
3,20230323A05KI1E,摩恩公司2023春季全国招商活动开始了!看看我能把与摩恩一起创造财富机会的送给那位有缘人! ...,促销广告、打折信息类,train.json,促销广告、打折信息类,提问求答鉴定类,汽车经销商新闻,好物推荐、直播预告类,跨洋购物,公司公告,汽车经销商新闻,合租&租房广告&售房广告,灌水水贴,跨洋购物,{'促销广告、打折信息类': '促销广告是指为了促进销售而发布的广告，通常包括打折、赠品、抽...
4,20230322A01G7Y1,露营季将至厂商既喜且忧:爆单抢单持续 价格战苗头已现财联社3月22日讯(记者 陈抗)五一露营...,促销广告、打折信息类,test.json,好物推荐、直播预告类,假冒仿制商品,促销广告、打折信息类,跨洋购物,股市、债市动态,公司财报,跨洋购物,促销广告、打折信息类,假冒仿制商品,汽车经销商新闻,{'好物推荐、直播预告类': '好物推荐是实质用户根据根据使用心得推荐一些优质的产品或服务，...


In [105]:
async def multi_label_evaluation(tools: AsyncioRequestGPT, data: dict, batch_number: int = 300):
    semaphore = asyncio.Semaphore(batch_number)
    tasks = [
        asyncio.create_task(tools.request_post(
            index=row['docid'],
            data={
                "possible_topic_info": row["possible_topic_info"],
                "content": row['content'],
                "label": list(row["possible_topic_info"].keys())
            }, semaphore=semaphore
        )) for index, row in data.iterrows()
    ]
    responses = [await f for f in tqdm(asyncio.as_completed(tasks), total=len(tasks))]

In [110]:
multi_label_evaluation_done = await multi_label_evaluation(multi_label_evaluation_tools, complete_recall_topic_df, semaphore_num)

100%|██████████| 965/965 [03:39<00:00,  4.39it/s]


In [109]:
multi_label_evaluation_tools.complete_result

[{'doc_id': '202205262392895',
  'llm_response': {'result': [{'topic_name': '便民信息',
     'topic_is_right': 'no',
     'explanation': '文章内容主要是关于旅游线路和旅行社服务的介绍，提供了西雅图周边游的景点和服务内容，属于旅游信息类内容，不符合便民信息的定义。因此，文章类型不正确。'}]}},
 {'doc_id': 'c069cb10d2e60a8e6aee82d5638803f6',
  'llm_response': {'result': [{'topic_name': '假冒仿制商品',
     'topic_is_right': 'yes',
     'explanation': '文章内容提到了百草味的夏威夷果奶油味500g产品，描述了产品的特点和优势，符合假冒仿制商品的定义，因此文章类型为假冒仿制商品是正确的。'},
    {'topic_name': '促销广告、打折信息类',
     'topic_is_right': 'yes',
     'explanation': '文章中提到了价格42.9元，以及赠送运费险的信息，属于促销广告、打折信息类的内容，因此文章类型为促销广告、打折信息类是正确的。'},
    {'topic_name': '股市、债市动态',
     'topic_is_right': 'no',
     'explanation': '文章内容与股市、债市动态无关，没有涉及大宗商品价格、期货行情等相关信息，因此不属于股市、债市动态，文章类型为股市、债市动态是错误的。'},
    {'topic_name': '跨洋购物',
     'topic_is_right': 'no',
     'explanation': '文章内容主要是关于百草味的夏威夷果奶油味500g产品的描述和推广，没有涉及跨国购物相关内容，因此不属于跨洋购物，文章类型为跨洋购物是错误的。'},
    {'topic_name': '好物推荐、直播预告类',
     'topic_is_right': 'no',
     'explanation': '文章内容主要是在推广特定产品，没有提供用户使用心得或

In [111]:
multi_label_evaluation_result = [(dic['doc_id'], topic_info["topic_name"], topic_info['topic_is_right'], topic_info['explanation']) for topic_info in  dic['llm_response']['result'] for dic in multi_label_evaluation_tools.complete_result if "llm_response" in dic and "result" in dic['llm_response'] and isinstance(dic['llm_response']['result'], list)]

NameError: name 'dic' is not defined

In [114]:
multi_label_evaluation_result = []
for dic in multi_label_evaluation_tools.complete_result:
    try:
        if "llm_response" in dic and "result" in dic['llm_response'] and isinstance(dic['llm_response']['result'], list):
            for topic_info in  dic['llm_response']['result']:
                multi_label_evaluation_result.append([
                    dic['doc_id'], topic_info["topic_name"], topic_info['topic_is_right'], topic_info['explanation']
                ])
    except:
        print(dic)


{'doc_id': '20230807A090CD9', 'llm_response': '{\n    "result": [\n        {\n            "topic_name":  "跨洋购物",\n            "topic_is_right": "no",\n            "explanation": "文章内容主要涉及银保渠道手续费率上涨、人身险企对抗、降低负债成本等内容，与跨洋购物无关，因此跨洋购物标签不正确。"\n        },\n        {\n            "topic_name":  "公司财报",\n            "topic_is_right": "no",\n            "explanation": "文章主要讨论银保渠道手续费率上涨、人身险企对抗、降低负债成本等内容，并未涉及具体公司的财务状况和经营业绩报告，因此公司财报标签不正确。"\n        },\n        {\n            "topic_name":  "促销广告、打折信息类",\n            "topic_is_right": "no",\n            "explanation": "文章内容讨论银保渠道手续费率上涨、人身险企对抗、降低负债成本，没有涉及促销广告、打折信息类内容，因此该标签不正确。"\n        },\n        {\n            "topic_name":  "公司公告",\n            "topic_is_right": "yes",\n            "explanation": "文章内容涉及保险公司与银行分公司的签约、监管部门要求控制负债成本、监管严令人身险企执行"                                                                                                    '}
